In [1]:
import tdt
import collections
import numpy as np
import sys
import seaborn as sns
import matplotlib.pylab as plt
# help(tdt.read_block)


In [2]:
TONE_DURATION = 0.25
IN_TONE_CAPTURE_START_OFFSET = 0.0
IN_TONE_CAPTURE_END_OFFSET = 0.2  # time between start offset and end offset will be counted
OUT_TONE_CAPTURE_START_OFFSET = 0.1
OUT_TONE_CAPTURE_END_OFFSET = 0.25 # time between start offset and end offset will be counted

TONES_PER_SECOND = 2

START_OFFSET = -9.1
END_OFFSET = 9.1

BLOCK_PATH = "/Users/smorgan/Documents/tdttmp/Block-1"
# BLOCK_PATH = "/Volumes/SM Research/Research Data/Simeon Morgan/111_140/2009-07-19 Auditory continuous 10s acclimatisation/2009-07-19 111_140/Block-1"

In [3]:
data = tdt.read_block(BLOCK_PATH, evtype=['epocs', 'snips', 'scalars'], nodata=1)

/Volumes/CaseSensitiveDevPartition/personal/tdt/.venv/lib/python3.8/site-packages/tdt/TDTbin2py.py:792: Warning: tnt file could not be processed
  warnings.warn('tnt file could not be processed', Warning)


read from t=0s to t=7457.32s


In [4]:
filtered = tdt.epoc_filter(data, "TriS", t=[START_OFFSET, END_OFFSET - START_OFFSET])

In [6]:
# Skip the last tone as the end offset should always go a little past the end
tones_to_include = int(((END_OFFSET - START_OFFSET) * TONES_PER_SECOND) - 1)

TIME_RANGE_ONSET_IDX = 0
TIME_RANGE_OFFSET_IDX = 1

SpikeCountRange = collections.namedtuple("SpikeCountRange", ("start", "end"))

all_trials_data = []
cspk_offset = 0
cspk_data = filtered["snips"]["CSPK"]
cspk_length = len(cspk_data["ts"])
np.set_printoptions(threshold=sys.maxsize)

for trial_number in range(len(filtered["time_ranges"][0])):
    in_tone_data = np.zeros((tones_to_include * 2, 32), np.uint)
    #out_tone_data = np.zeros((tones_to_include, 32), np.uint)
    for tone_number, tone_offset in enumerate(filter(lambda x: filtered["time_ranges"][TIME_RANGE_ONSET_IDX][trial_number] < x < filtered["time_ranges"][TIME_RANGE_OFFSET_IDX][trial_number], filtered["epocs"]["StiS"]["onset"])):
        if tone_number >= tones_to_include:
            continue

        spike_cumulator_range = SpikeCountRange(start=tone_offset+IN_TONE_CAPTURE_START_OFFSET, end=tone_offset+IN_TONE_CAPTURE_END_OFFSET)
        while cspk_data["ts"][cspk_offset] < spike_cumulator_range.start:
            cspk_offset += 1

        while cspk_data["ts"][cspk_offset] < spike_cumulator_range.end:
            in_tone_data[tone_number * 2, cspk_data["chan"][cspk_offset][0]-1] += 1
            cspk_offset += 1
            
        spike_cumulator_range = SpikeCountRange(start=tone_offset+TONE_DURATION+OUT_TONE_CAPTURE_START_OFFSET, end=tone_offset+TONE_DURATION+OUT_TONE_CAPTURE_END_OFFSET)
        while cspk_data["ts"][cspk_offset] < spike_cumulator_range.start:
            cspk_offset += 1

        while cspk_data["ts"][cspk_offset] < spike_cumulator_range.end:
            in_tone_data[tone_number * 2 + 1, cspk_data["chan"][cspk_offset][0]-1] += 1
            cspk_offset += 1


#     print(in_tone_data)
#     plt.rcParams['figure.figsize'] = [100/2.54, 80/2.54]
#     plt.imshow(in_tone_data, cmap='hot', interpolation='nearest')
#     plt.show()
#     ax = sns.heatmap(in_tone_data, linewidth=0.5)
#     plt.show()
#     print("Out tone data")
#     print(out_tone_data)

In [15]:
filtered["epocs"]["StiS"]["offset"][0] - filtered["epocs"]["StiS"]["onset"][0]

0.2500198399999931

In [16]:
filtered["epocs"]

RefC	[struct]
ReBM	[struct]
StBM	[struct]
SweS	[struct]
TriS	[struct]
StiS	[struct]
BloS	[struct]
Tick	[struct]
Chan	[struct]
Attn	[struct]
Samp	[struct]
Freq	[struct]
AFrq	[struct]
Curr	[struct]

In [38]:
list(enumerate(zip(filtered["time_ranges"][0], filtered["time_ranges"][1])))

[(0, (112.48359552000001, 130.68359552)),
 (1, (234.48242304000001, 252.68242304)),
 (2, (374.48106112, 392.68106112)),
 (3, (500.47984768, 518.67984768)),
 (4, (643.4784576, 661.6784576)),
 (5, (796.47699072, 814.67699072)),
 (6, (942.47558272, 960.6755827200001)),
 (7, (1046.47455872, 1064.67455872)),
 (8, (1234.47274624, 1252.6727462400002)),
 (9, (1402.47113856, 1420.6711385600001)),
 (10, (1567.46953856, 1585.66953856)),
 (11, (1701.4682432000002, 1719.6682432000002)),
 (12, (1847.4668556800002, 1865.6668556800003)),
 (13, (1955.46580096, 1973.66580096)),
 (14, (2124.46416, 2142.66416)),
 (15, (2252.46295168, 2270.66295168)),
 (16, (2407.46142848, 2425.66142848)),
 (17, (2539.46015872, 2557.66015872)),
 (18, (2641.45917056, 2659.65917056)),
 (19, (2780.4578112, 2798.6578111999997)),
 (20, (2933.4563545600004, 2951.65635456)),
 (21, (3123.45451648, 3141.65451648)),
 (22, (3281.45297536, 3299.65297536)),
 (23, (3451.45139328, 3469.65139328)),
 (24, (3574.45016704, 3592.65016704)),
 